In [1]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import umap.umap_ as umap
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Importing data

In [2]:
raw_counts_df = pd.read_csv('/Users/johnsmith/Desktop/Senior Design Stuff/tcga.brca.rnaseq.unstranded.raw.counts.matrix-2.txt', sep='\t', index_col=0)

In [3]:
metadata = pd.read_csv('/Users/johnsmith/Desktop/Senior Design Stuff/tcga_metadata_poore_et_al_2020_1Aug23.csv')

In [4]:
tpm = pd.read_csv("/Users/johnsmith/Desktop/Senior Design Stuff/tpm.csv")

In [99]:
#tpm

In [6]:
#filtering out non count columns
tpm_filt = tpm.iloc[:,1:]
tpm_filt


,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
0,64.3002,0.3433,77.0779,9.9974,4.1080,10.9674,21.9577,39.5030,9.3605,38.5854,...,0,0.0000,0.6935,0,0.0000,0.0000,10.5998,0,0.0527,0.6032
1,79.8321,1.0345,231.9536,5.4463,3.0821,3.1379,5.8744,36.4695,6.7241,42.4712,...,0,0.0000,0.1608,0,0.0000,0.0000,37.0966,0,0.1032,0.4090
2,15.6522,0.3055,82.2766,8.9320,2.3835,3.1853,25.2157,48.8798,5.0582,20.4705,...,0,0.0000,0.0528,0,0.0000,0.0000,7.7362,0,0.0000,0.9544
3,33.1976,0.0000,116.4500,4.2842,9.0514,3.4080,9.3476,45.1569,5.3149,34.3415,...,0,0.0000,0.2276,0,0.0000,0.0000,12.0389,0,0.0415,0.1188
4,43.9791,1.1406,117.9486,15.2048,8.3656,11.7971,30.6031,43.2496,16.0961,55.8942,...,0,0.0000,0.3742,0,0.0000,0.0000,18.3220,0,0.0642,0.9353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,21.6081,2.2469,71.0683,31.3386,9.9318,2.6636,16.3486,37.3209,15.0655,60.8455,...,0,0.0000,0.6213,0,2.2041,0.0225,34.9953,0,0.0373,0.7469
1227,29.5505,0.3182,116.5785,27.5464,11.2577,25.7744,29.3096,42.5390,12.4963,45.2037,...,0,0.7828,0.1483,0,0.0000,0.0000,9.3048,0,0.0195,0.2796
1228,35.9726,0.4839,123.1551,13.0063,5.9423,7.2981,20.3680,58.0744,15.0108,35.1108,...,0,0.0000,0.0418,0,0.0000,0.0000,13.2200,0,0.0412,0.5196
1229,115.3770,0.3965,127.0776,15.3380,23.0260,18.1687,5.3974,36.6013,25.6138,61.3307,...,0,0.0000,0.2978,0,0.0000,0.0000,8.6145,0,0.0122,0.5225


In [7]:
metadata_filt = metadata[(metadata['investigation'].str.contains('BRCA', na=False)) & (metadata['experimental_strategy'].str.contains('RNA-Seq', na=False))]

In [8]:
metadata_filt.reset_index(inplace=True)

In [30]:
#metadata_filt

In [12]:
#raw_counts_df['Sample ID'].str.split('-').str[3].unique()
#aw_counts_df

In [103]:
#filtering columns
columns_to_drop = ['Sample ID','Case ID', 'Sample Type']
raw_counts_df_filtered = raw_counts_df.drop(columns=[col for col in columns_to_drop if col in raw_counts_df.columns])
raw_counts_df_primary = raw_counts_df[raw_counts_df['Sample Type'] == 'Primary Tumor'].drop(columns=[col for col in columns_to_drop if col in raw_counts_df.columns])

In [105]:
raw_counts_df_primary.reset_index(inplace=True)

In [81]:
raw_counts_df['Sample Type'].unique()

array(['Primary Tumor', 'Solid Tissue Normal', 'Metastatic'], dtype=object)

In [34]:
raw_counts_df_filtered

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AL451106.1,AC092910.4,AC073611.1,AC136977.1,AC078856.1,AC008763.4,AL592295.6,AC006486.3,AL391628.1,AP006621.6
0,324,0,6231,661,695,333,2907,1663,3538,5140,...,0,0,7,0,0,0,227,0,0,15
1,4241,13,3226,1812,1306,591,4209,6056,2643,4666,...,0,0,34,0,0,1,441,0,5,45
2,3573,770,1607,1040,215,573,13487,2373,3212,1397,...,0,1,16,0,1,1,654,0,22,11
3,3627,176,1810,1598,573,676,4147,2548,1701,1908,...,0,0,8,0,0,0,318,0,3,23
4,292,9,2038,2101,447,639,2327,2704,3900,2952,...,0,0,11,0,0,0,431,0,3,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,1383,50,2136,1396,543,264,2399,1555,1098,2029,...,0,0,28,0,0,0,554,0,3,7
1227,2377,33,2746,3120,954,427,2896,2860,1917,3655,...,0,0,19,0,0,0,357,0,8,34
1228,7874,22,3951,3720,1352,848,3513,5645,4802,9290,...,0,0,43,0,0,0,778,0,18,27
1229,4169,44,1033,1163,296,293,1487,1387,1310,2416,...,0,0,21,0,2,0,272,0,10,29


# UMAP of raw data

In [106]:
# Scale the data
scaler = StandardScaler()
gene_expression_scaled = scaler.fit_transform(raw_counts_df_primary)

# Perform UMAP
umap_reducer = umap.UMAP(n_components=2, random_state=42)
umap_embedding = umap_reducer.fit_transform(gene_expression_scaled)

# Prepare a dataframe for visualization
primary = raw_counts_df[raw_counts_df['Sample Type'] == 'Primary Tumor']
umap_df_primary = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df_primary['Sample ID'] = primary['Sample ID']  # Add Sample ID back for labeling
umap_df_primary['Sample Type'] = primary['Sample Type']
umap_df_primary['TSS'] = umap_df_primary['Sample ID'].str.split('-').str[1]

# Create a unique color map for the 40 classes
unique_classes = umap_df_primary['TSS'].unique()
custom_color_map = {cls: px.colors.qualitative.Alphabet[i % len(px.colors.qualitative.Alphabet)] 
                    for i, cls in enumerate(unique_classes)}



/Users/johnsmith/miniconda3/envs/HRDclass/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [107]:
# Use the viridis palette for categorical data

fig = px.scatter(
    umap_df_primary,
    x='UMAP1',
    y='UMAP2',
    color='TSS',
    title="UMAP projection of Raw Primary Tumor samples colored by Tissue Source",
    labels={'UMAP1': 'UMAP Dimension 1', 'UMAP2': 'UMAP Dimension 2'},
    color_discrete_map=custom_color_map
)

fig.show()

: 

In [82]:
# Scale the data
scaler = StandardScaler()
gene_expression_scaled = scaler.fit_transform(raw_counts_df_filtered)

# Perform UMAP
umap_reducer = umap.UMAP(n_components=2, random_state=42)
umap_embedding = umap_reducer.fit_transform(gene_expression_scaled)

# Prepare a dataframe for visualization
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['Sample ID'] = raw_counts_df['Sample ID']  # Add Sample ID back for labeling
umap_df['Sample Type'] = raw_counts_df['Sample Type']

/Users/johnsmith/miniconda3/envs/HRDclass/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [83]:
umap_df['TSS'] = umap_df['Sample ID'].str.split('-').str[1]

In [96]:
# Create a unique color map for the 40 classes
unique_classes = umap_df['TSS'].unique()
custom_color_map = {cls: px.colors.qualitative.Alphabet[i % len(px.colors.qualitative.Alphabet)] 
                    for i, cls in enumerate(unique_classes)}

# Use the viridis palette for categorical data

fig = px.scatter(
    umap_df,
    x='UMAP1',
    y='UMAP2',
    color='TSS',
    title="UMAP projection of Raw data colored by Tissue Source",
    labels={'UMAP1': 'UMAP Dimension 1', 'UMAP2': 'UMAP Dimension 2'},
    color_discrete_map=custom_color_map
)

fig.show()

In [97]:
fig = px.scatter(
    umap_df,
    x='UMAP1',
    y='UMAP2',
    color='Sample Type',
    title="UMAP projection of Raw data colored by Sample Type",
    labels={'UMAP1': 'UMAP Dimension 1', 'UMAP2': 'UMAP Dimension 2'},
    color_discrete_map=custom_color_map
)

fig.show()

# PCA for raw data

In [85]:
# Perform PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(gene_expression_scaled)

# Create a dataframe for the PCA results
# pca_df = pd.DataFrame(pca_result, columns=['PC1','PC2', 'PC3','PC4', 'PC5','PC6'])
pca_df = pd.DataFrame(pca_result, columns=['PC1','PC2'])
pca_df['Sample ID'] = raw_counts_df['Sample ID']  # Add Sample ID for hover info

In [86]:
pca_df['ID'] = pca_df['Sample ID'].str.split('-').str[1]

In [87]:
# Visualize PCA results with Plotly
fig = px.scatter(
    pca_df,
    x='PC1',
    y='PC2',
    color = 'ID',
    title="PCA Analysis of RNA-seq Data",
    labels={'PC2': 'Principal Component 2', 'PC4': 'Principal Component 4'},
    template="plotly_white"
)
fig.update_traces(marker=dict(size=8, opacity=0.7))
fig.show()

# Optional: Print explained variance ratio
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

Explained Variance Ratio: [0.11042835 0.05724225]


# t-SNE for Raw data

In [88]:
# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_result = tsne.fit_transform(gene_expression_scaled)

# Create a DataFrame for t-SNE results
tsne_df = pd.DataFrame(tsne_result, columns=['t-SNE1', 't-SNE2'])
tsne_df['Sample ID'] = raw_counts_df['Sample ID']  # Add Sample ID for hover info

In [89]:
tsne_df['TSS'] = tsne_df['Sample ID'].str.split('-').str[1]

In [91]:
# Visualize t-SNE results using Plotly
fig = px.scatter(
    tsne_df,
    x='t-SNE1',
    y='t-SNE2',
    color = 'TSS',
    title="t-SNE Analysis of RNA-seq Data",
    labels={'t-SNE1': 't-SNE Component 1', 't-SNE2': 't-SNE Component 2'},
    template="plotly_white"
)
fig.update_traces(marker=dict(size=8, opacity=0.7))
fig.show()

# UMAP for tpm data

In [ ]:
# Scale the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(tpm_filt)

# Perform UMAP
umap_reducer = umap.UMAP(n_components=2, random_state=42)
umap_embedding = umap_reducer.fit_transform(data_scaled)


/Users/johnsmith/miniconda3/envs/HRDclass/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [93]:
# Plot the UMAP embedding
umap_df_tpm = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df_tpm['TSS'] = tpm['Unnamed: 0'].str.split('-').str[1]
umap_df['Sample Type'] = tpm['Sample Type']

In [94]:

# Create a unique color map for the 40 classes
unique_classes = umap_df_tpm['TSS'].unique()
custom_color_map = {cls: px.colors.qualitative.Alphabet[i % len(px.colors.qualitative.Alphabet)] 
                    for i, cls in enumerate(unique_classes)}

# Use the viridis palette for categorical data

fig = px.scatter(
    umap_df_tpm,
    x='UMAP1',
    y='UMAP2',
    color='TSS',
    title="UMAP projection of TPM data colored by Tissue Source",
    labels={'UMAP1': 'UMAP Dimension 1', 'UMAP2': 'UMAP Dimension 2'},
    color_discrete_map=custom_color_map
)

fig.show()

In [98]:
fig = px.scatter(
    umap_df_tpm,
    x='UMAP1',
    y='UMAP2',
    color='Sample Type',
    title="UMAP projection of TPM data colored by Sample Type",
    labels={'UMAP1': 'UMAP Dimension 1', 'UMAP2': 'UMAP Dimension 2'},
    color_discrete_map=custom_color_map
)

fig.show()

ValueError: Value of 'color' is not the name of a column in 'data_frame'. Expected one of ['UMAP1', 'UMAP2', 'TSS'] but received: Sample Type